In [ ]:
import pandas as pd
import numpy as np
import time

In [ ]:
#read in the list of videos contained in the csv
df = pd.read_csv('vidz.csv')

In [ ]:
#drop columns with redundant information
df = df.drop(columns=['comments_disabled', 'ratings_disabled' ,'thumbnail_link'])

In [ ]:
df.categoryId = df.categoryId.astype('object')

In [ ]:
df.info()

## Convert to datetime --> ppublised at, trending date, duration (timedelta)

In [ ]:
#convert publishedAT to datetime column
df.publishedAt = pd.to_datetime(df.publishedAt, utc=True)
df.publishedAt

In [ ]:
#set date to next day in order to capture videos released at different times in different time zones
df.trending_date = '22.28.10'

In [ ]:
#convert trending to datetime column
df.trending_date = pd.to_datetime(df.trending_date, format='%y.%d.%m', utc=True)
df.trending_date

In [ ]:
#import tzinfo inorder to strip time zone information from published at. 
#this makes it a "naive" datetime object. may want to change this approach
# from datetime import tzinfo

# df.loc[1,'publishedAt'].replace(tzinfo=None)
#strips the timezone from each row
# for n in range(0,200):
#     df.loc[n,'publishedAt']= df.loc[n,'publishedAt'].replace(tzinfo=None)
# df.loc[10, ['publishedAt']]

In [ ]:
#create age column. may want to give timezone info to trending date instead of removing it from pblishedAt
df['age']=(df.trending_date - df.publishedAt)

In [ ]:
df.age.sort_values().head(10)

## View:Like ratio that can score the video | view:comment ratio

Have them all as a weighted ratio

df['engagement'] = (df.likes + df.comment_count * 4 )/df.view_count

In [ ]:
#creates engagement metric. not sure how to do weightd columns really
#df['engagement'] = (df.view_count - df.likes) + (df.likes * 2) + (df.comment_count * 4) 
df['engagement'] = (df.likes + df.comment_count * 4 )/df.view_count

In [ ]:
df.engagement

## Is the video sponsored?


In [ ]:
#adds sponsored column based on appearance of word sponsored in the description
df['sponsored'] = np.where(df.description.str.contains('sponsor'), 1, 0)
df[df.description.str.contains('sponsored')].shape

## Video language


## Percent of capital letters in title 


## Age restricted


## When we talk about subscribers we can take a look at age of channel


## How to drive up subscribers is a slightly separate question but we can ask it


## At what point does the video view count pass the subscriber view count


## Let’s think about tags and how many words they have in common with the descriptions


In [ ]:
#countes number of tags given to video BEFORE stripping out extraneous things
df['num_of_tags'] = df.tags.str.split('|').str.len()

In [ ]:
#gets rid of separator
df.tags = df.tags.str.replace('|'," ")

In [ ]:
#Imports spacy
import spacy

In [ ]:
#creates the nlp object that is going to do the heavy lifting
nlp = spacy.load("en_core_web_sm")

In [ ]:
#uses the nlp object to convert the input text into a doc
doc = nlp(df.loc[0].tags)

In [ ]:
#goes through tokens (words) in each doc
for token in doc:
    # Get the token text, part-of-speech tag and dependency label
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # This is for formatting only
    print(f"{token_text:<12}{token_pos:<10}{token_dep:<10}")

# Iterate over the predicted entities
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

In [ ]:
#creates set of unique words in doc 
tokens = set()
for token in doc:
    tokens.add(token.text)
print(tokens)

In [ ]:
docb = nlp(df.loc[4].description)

In [ ]:
for token in docb:
    # Get the token text, part-of-speech tag and dependency label
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # This is for formatting only
    print(f"{token_text:<12}{token_pos:<10}{token_dep:<10}")

# Iterate over the predicted entities
for ent in docb.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

In [ ]:
tokens = set()
for token in docb:
    if token.pos_ not in ['SYM', 'PUNCT', 'DET']:
        print(token.pos_)
        tokens.add(token.text)

for ent in docb.ents:
    print(ent.text, ent.label_)

In [ ]:
for ent in docb.ents:
    print(ent.text, ent.label_)

In [ ]:
def clean_text(text):
    """ 
    Purpose:
        to clean text input into function by removing duplicate words, punctuations, and other things
    ---
    Parameters:
        text: a string
    ---
    Returns:
        tokens: a set of words found in the input text
    """

    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    tokens = set()
    
    for token in doc:
        if token.pos_ not in ['SYM', 'PUNCT', 'DET']:
            tokens.add(token.text)

    for ent in doc.ents:
        tokens.add(ent.text)

    return tokens
    
    

In [ ]:
df['cleaned_tags'] =  df['description'].apply(lambda x: clean_text(x))

In [ ]:
df['cleaned_desc'] = df['description'].apply(lambda x: clean_text(x))

## Think about combining the countries top 25 lists and control for duplicates. 
* This way we can classify what videos have been a top 25 video 


## We can target placing videos in/out of the top 25 


In [ ]:
#create rank and top 25 categories

In [ ]:
df['rank'] = df.index + 1

In [ ]:
df['top_25'] = np.where(df['rank'] < 26, 1, 0)

In [ ]:
df.top_25

## Convert Duration

In [ ]:
df.duration.head(20)

In [ ]:
testy = 'PT3M45S'

In [ ]:
testy.__contains__('M')

In [ ]:
def clean_duration(duration):

    if 'S' not in duration:
        duration += '00S'

    if duration.__contains__('M'):
        duration = int(duration.split('M')[0].split('T')[1]) * 60 + int(duration.split('M')[1][:-1])
    else:
        duration = int(duration[-3:-1])

    return duration

In [ ]:
df['duration'] = df['duration'].apply(lambda x:clean_duration(x))

In [ ]:
df['duration']

In [ ]:
df.info()

# Exploration

In [ ]:
df.engagement.plot.box()

In [ ]:
df[df.top_25 == 1].engagement.plot.box()

In [ ]:
df[df.top_25 != 1].engagement.plot.box()

## Takeaways
* engagement higher with top 25 videos

In [ ]:
import pandas as pd
df = pd.read_pickle('prepared.pkl')
df["channelId"] = np.nan
df

In [ ]:
df.info()

In [ ]:
vid

In [ ]:
df.video_id.to_csv('video_ids.txt',index=False)

In [ ]:
one_df = df.copy()

In [ ]:
one_df = one_df.drop_duplicates('channelTitle')

In [ ]:
one_df.video_id.to_csv('video_ids.txt',index=False)

In [ ]:
import os
import glob
import pandas as pd
#set working directory
os.chdir("/Users/vincentbanuelos/codeup-data-science/top_200/channeloutput")

#find all csv files in the folder
#use glob pattern matching -> extension = 'csv'
#save result in list -> all_filenames
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#print(all_filenames)

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_channel.csv", index=False, encoding='utf-8-sig')

In [1]:
from googletrans import Translator

translator = Translator()

In [13]:
result = translator.translate('아이들아는형님')

In [14]:
print(result.src)
print(result.dest)
print(result.origin)
print(result.text)
print(result.pronunciation)

ko
en
아이들아는형님
Children who know children
None


In [17]:
import pandas as pd
df = pd.read_csv('youtube_tren.csv',index_col=0)
df

,video_id,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,comment_count,...,num_of_tags,word_bank,cleaned_tags,cleaned_desc,title_in_description,title_in_tags,pct_tags_in_description,title_lengths,desc_lengths,tags_length
0,sWwHWK7OYJ8,[ENG][아형✪하이라이트] (여자)아이들 완전체 떴다↗ 멋짐 폭발♥ 5인 5색 토...,2022-10-29 13:26:44+00:00,아는형님 Knowingbros,Entertainment,2022-11-02 00:00:00+00:00,아는 형님 형님 형님 학교 미연 민니 소연 우기 슈화 아이들 여자아이들 강호동 이수...,1098919,19090,861,...,67,"['[', '아형', '✪', '하이라이트', '여자)아이들', '완전체', '떴다...","{'여자아이들', '아이들아는형님', 'MC', '전소연아는형님', '인기가요', ...","{'#여자아이들 #', '여자아이들', '아형', '5인', '리플레이', '수',...",0,0,0.013889,79,200,325
1,4xl9KfUg8Lc,Pathaan | Official Teaser | Shah Rukh Khan | D...,2022-11-02 05:31:19+00:00,YRF,Film_Animation,2022-11-02 00:00:00+00:00,shah rukh khan shahrukh khan srk shah rukh kha...,7638460,764869,75586,...,23,"['special', 'surprise', 'for', 'very', 'specia...","{'john abraham', 'john', 'padukone', 'bollywoo...","{'YRF50', 'JollyVisual', 'Pathaan', 'Manas', '...",0,0,0.013889,94,1454,459
2,vzxI6U_l72g,ヒカルくんとようやく戦えるときが来ました。,2022-10-31 10:30:10+00:00,ロードシルク,People_Blogs,2022-11-02 00:00:00+00:00,シルクロード ロードシルク Fischer’s フィッシャーズ 個人 チャンネル 個チャン ...,613365,15756,866,...,32,"[""ヒカルくんhttps://www.youtube.com/channel/UCaminw...","{'大乱闘', 'ガチ', '対戦', '趣味', 'Fischer', 'サブカル', '...",{'業界の皆様・ご依頼の方々・その他ご相談がある方は、UUUMウェブサイトにある「サービス」...,0,0,0.013889,21,717,148
3,AcznL5daQvA,Fear PIRATE CHAMPION's Spear! Clash of Clans S...,2022-11-01 08:00:04+00:00,Clash of Clans,Gaming,2022-11-02 00:00:00+00:00,clash of clans COC Clash of Clans Gameplay Cla...,2258144,44366,938,...,28,"['Ahoy', '⚓', 'new', 'month', 'of', 'Season', ...","{'clasharama', 'challenges', 'Hall', 'Clash', ...","{'years', 'Also', 'battle', 'policy', 'http://...",0,0,0.013889,62,1874,444
4,ZYO1Z4Ajdvw,ЗА МНОЙ СЛЕДИЛИ?! ТОТ САМЫЙ ЛЕС В ЯПОНИИ! Ghos...,2022-10-31 20:54:41+00:00,Дима Масленников (Pognali Show),Entertainment,2022-11-02 00:00:00+00:00,ЗА МНОЙ СЛЕДИЛИ?! ТОТ САМЫЙ ЛЕС В ЯПОНИИ ТОТ С...,3552774,325414,14886,...,32,"['Оформи', 'подписку', 'на', 'VK', 'Музыку', '...","{'камеру', 'МНОЙ', 'ghostbusters', 'жесть', 'п...","{'ЯПОНСКОМ', 'GhostBuster Аналитика', 'что реб...",0,0,0.013889,63,1132,446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2048,iDBRKLjfJS4,망한 모텔 사서 한 달에 4천만원 버는 38살 사장님,2022-10-09 09:34:15+00:00,탐구생활 - 돈이 되는 삶의 이야기,People_Blogs,2022-11-02 00:00:00+00:00,[none],1837172,12016,1015,...,1,"['광주와', '담양에서', '호텔을', '운영하는', '여인욱', '대표님입니다탐...","{'none', '['}","{'tamgu-life@naver.com', '대표님입니다탐구생활', '여인욱', ...",0,0,0.000000,29,54,0
2049,U26zXbioeMs,"Vete a la Versh - T5, E16: La Leyenda de Melda...",2022-10-21 21:36:39+00:00,Darkar Company Studios,Film_Animation,2022-11-02 00:00:00+00:00,melda darkar mecoboy zelda versh nintendo mario,816003,103955,6399,...,7,"['Gracias', 'por', 'apoyar', 'el', 'proyecto',...","{'melda', 'zelda', 'mario', 'mecoboy', 'darkar...","{'http://www.twitter.com', 'el', 'apoyar', 'mi...",0,0,0.000000,60,224,47
2050,WRr0EDTrDx8,【プロ】バレー第1回目のゲストが本気すぎたんだけど、、、,2022-10-23 11:00:11+00:00,スカイピース,People_Blogs,2022-11-02 00:00:00+00:00,スカイピース,1977549,48825,3060,...,1,"['\u200b', '\u200b@コムドット', ' ', '@中町兄妹', ' ', ...",{'スカイピース'},{'https://www.youtube.com/channel/UC5VZjrV5x9J...,0,0,0.000000,28,1517,6
2051,j-3Tpmi7CCk,"Pabllo Vittar, Gloria Groove - AMEIANOITE (Off...",2022-10-21 02:59:03+00:00,Pabllo Vittar,Music,2022-11-02 00:00:00+00:00,[none],1529830,114260,11625,...,1,"['Watch', 'more', 'Pabllo', 'Vittar', 'YouTube...","{'none', '['}","{'YouTube', 'vídeos', 'Suscríbete', 'sociais!h...",0,0,0.000000,64,352,0
